# 鏡頭校正

# 圖片偵測

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import importlib
import cube_detector as CD
import cube_detector2 as CD2
import pickle

model = YOLO("yolov8n-seg-custom.pt")
surface_model = YOLO('cube_surface_seg2.pt')


In [ ]:
def draw_axis(img, corners, image_points):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(np.intp(image_points[0].ravel())), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(np.intp(image_points[1].ravel())), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(np.intp(image_points[2].ravel())), (0,0,255), 5)
    return img
def draw_xy_lines(img, image_points):
    img = cv2.line(img, tuple(np.intp(image_points[0].ravel())), tuple(np.intp(image_points[1].ravel())), (0,0,0), 5)
    img = cv2.line(img, tuple(np.intp(image_points[1].ravel())), tuple(np.intp(image_points[2].ravel())), (0,0,0), 5)
    return img
def draw_z_lines(img, image_points):
    img = cv2.line(img, tuple(np.intp(image_points[1].ravel())), tuple(np.intp(image_points[3].ravel())), (0,0,255), 5)
    # img = cv2.line(img, tuple(np.intp(image_points[1].ravel())), tuple(np.intp(image_points[2].ravel())), (0,0,0), 5)
    return img

In [ ]:
importlib.reload(CD)
importlib.reload(CD2)

detector=CD.cubeDetector(model,isCudaSupport=False,processing_hegiht=300)
detector2=CD2.cubeDetector(model,surface_model,isCudaSupport=False,processing_hegiht=150)

# 側鏡頭PNP

In [27]:
# img = cv2.imread("test1.jpg")
frame = cv2.imread("pnp_test_img/img2.png")
detector.isCudaSupport=True
height =480
scale = height / frame.shape[0]
frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
img2  = detector2.detect(frame,index=None,color=None,show_process_img=True)

cube_imagePoints=detector2.get_cube_largest_surface_imagePoints("yellow")

cube_objectPoints=np.array([(0,0,0),(0,25,0),(25,25,0),(25,0,0),(25,0,-25),(0,0,-25)])

with open('camera/calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)
with open('table_points.pkl','rb') as file:
    table_objectPoints,table_imagePoints = pickle.load(file)
with open('table_solvePnP.pkl','rb') as file:
    table_rvec,table_tvec = pickle.load(file)
# print(cube_imagePoints)
PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints.astype(float)[:len(cube_imagePoints)],cube_imagePoints,mtx,dist)
# ret,table_rvecs,table_tvecs = cv2.solvePnP(table_objectPoints,table_imagePoints,mtx,dist)

axis = np.float32([[25,0,0], [0,25,0], [0,0,25]]).reshape(-1,3)
axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
img2=draw_axis(img2,cube_imagePoints.astype(int),axis_cube_img_points)
img2=draw_axis(img2,table_imagePoints.astype(int),axis_table_img_points)
if PnP_success:
    R1, _ = cv2.Rodrigues(cube_rvec)
    R2, _ = cv2.Rodrigues(table_rvec)
    T1 = np.eye(4) 
    T2 = np.eye(4) 
    T1[:3, :3], T1[:3, 3] = R1, cube_tvec.T  # 方塊座標系旋轉加平移到相機座標系
    T2[:3, :3], T2[:3, 3] = R2, table_tvec.T # 桌面座標系旋轉加平移到相機座標系
    T2=np.linalg.inv(T2) # 取逆矩陣，變成 相譏座標系到桌面座標系
    transform_matrix = T2 @ T1  
    transformed_point = transform_matrix @ np.array([[12.5, 12.5, -12.5, 1]]).T
    # 输出结果
    print("物體在桌面座標系下的座標:\n", transformed_point)

x,y = transformed_point[0:2].flatten()
xy_line_points=np.float32([[0,y,0],[x,y,0],[x,0,0]]).reshape(-1,3)
xy_line_points_img,_=cv2.projectPoints(xy_line_points,table_rvec,table_tvec,mtx,dist)
img2 = draw_xy_lines(img2,xy_line_points_img)
cv2.imshow("test", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 480x640 3 cubes, 115.5ms
Speed: 50.3ms preprocess, 115.5ms inference, 29.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 surfaces, 114.2ms
Speed: 16.8ms preprocess, 114.2ms inference, 34.4ms postprocess per image at shape (1, 3, 480, 640)



[[[352 370]]

 [[287 368]]]
[2]
1
[1]


0: 480x640 7 surfaces, 123.5ms
Speed: 14.8ms preprocess, 123.5ms inference, 29.7ms postprocess per image at shape (1, 3, 480, 640)



[[[507 305]]

 [[444 302]]]
[1]
1
[1]


0: 480x640 7 surfaces, 120.6ms
Speed: 16.3ms preprocess, 120.6ms inference, 35.4ms postprocess per image at shape (1, 3, 480, 640)


物體在桌面座標系下的座標:
 [[     101.26]
 [    -14.888]
 [     84.587]
 [          1]]


# 夾抓鏡頭PNP

# 測試

## 鏡頭拍照

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

num = 101

while cap.isOpened():

    succes, frame = cap.read()

    k = cv2.waitKey(5)

    if k == ord('q'):
        break
    elif k == ord('s'): # wait for 's' key to save and exit
        cv2.imwrite('pnp_test_img/img' + str(num) + '.png', frame)
        # cv2.imwrite('train_img/cube(' + str(num) + ').JPG', frame)
        print("image saved!")
        num += 1

    cv2.imshow('Img',frame)

# Release and destroy all windows before termination
cap.release()

cv2.destroyAllWindows()

## 模型測試

In [ ]:
import cube_detector2 as CD2
# frame = cv2.imread("pnp_test_img/img11.png")
frame = cv2.imread("test7.jpg")
detector.isCudaSupport=True
height =480
scale = height / frame.shape[0]
frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
img2  = detector2.detect(frame,index=None,color=None,show_process_img=True)
cv2.imshow("img2",img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 其他測試

In [ ]:
import numpy as np

A = np.array([[1, 2,0],
              [3, 4,0],
              [0,0,1]])

B = np.array([[1, 0,2],
              [0, 1,2],
              [0,0,1]])

result =  B @A
print(result)

## real time test

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
cap.release()

### 實際用

In [ ]:


with open('camera/calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)
with open('table_points.pkl','rb') as file:
    table_objectPoints,table_imagePoints = pickle.load(file)
with open('table_solvePnP.pkl','rb') as file:
    table_rvec,table_tvec = pickle.load(file)
color = "green"
while cap.isOpened():
    cap_success, frame = cap.read()
    if cap_success:
        height =480
        scale = height / frame.shape[0]
        frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        img2  = detector2.detect(frame,index=None,color=None,show_process_img=True)

        cube_imagePoints=detector2.get_cube_largest_surface_imagePoints(color)
        if cube_imagePoints is None:
            cv2.imshow("result",frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            continue
        cube_objectPoints=np.array([(0,0,0),(0,25,0),(25,25,0),(25,0,0)])

        try_counter = 0

        radius = int(0.04*cv2.arcLength(contour_outer,True))
        vector = cube_imagePoints[2,:]-cube_imagePoints[0,:]
        unit_vector = vector/np.linalg.norm(vector)

        contour_outer =detector2.get_cube_contour_outer(color)
        M = cv2.moments(contour_outer)
        if M["m00"] != 0:
            centroid_X = int(M["m10"] / M["m00"])# 算形心x
            centroid_Y = int(M["m01"] / M["m00"])# 算形心y
        
        is_predict_fail = False
        while True:
            PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints.astype(float),cube_imagePoints,mtx,dist)

            if PnP_success:
                R1, _ = cv2.Rodrigues(cube_rvec)
                R2, _ = cv2.Rodrigues(table_rvec)
                T1 = np.eye(4) 
                T2 = np.eye(4) 
                T1[:3, :3], T1[:3, 3] = R1, cube_tvec.T  # 方塊座標系旋轉加平移到相機座標系
                T2[:3, :3], T2[:3, 3] = R2, table_tvec.T # 桌面座標系旋轉加平移到相機座標系
                T2=np.linalg.inv(T2) # 取逆矩陣，變成 相譏座標系到桌面座標系
                transform_matrix = T2 @ T1  
                transformed_point = transform_matrix @ np.array([[12.5, 12.5, -12.5, 1]]).T
                # 输出结果
                # print("物體在桌面座標系下的座標:\n", transformed_point)
            
            x,y,z = transformed_point[0:3].flatten()
            predict_cube_center_imagePoint = cv2.projectPoints(np.float32([x,y,z]),table_rvec,table_tvec,mtx,dist)
            predict_cube_center_imagePoint = predict_cube_center_imagePoint[0].astype(np.int16)
            if np.any(predict_cube_center_imagePoint>img2.shape[1]):
                is_predict_fail = True
                print("predict fail")
                break
            # print(np.linalg.norm(np.array((centroid_X,centroid_Y))-predict_cube_center_imagePoint))
            if np.linalg.norm(np.array((centroid_X,centroid_Y))-predict_cube_center_imagePoint)<=radius:
                # print(z,"-------------------------------------------------")

                break
            
            # 錯誤修正嘗試
            pieces = 40
            delta = radius/pieces
            if np.any(np.isnan(unit_vector)):
                is_predict_fail = True
                print("predict fail")
                break

            # print(try_counter,unit_vector)
            if try_counter<pieces:
                cube_imagePoints[2,:] += delta*unit_vector
            elif pieces <=try_counter <2*pieces:
                if try_counter == pieces:
                    cube_imagePoints[2,:] -= pieces*delta*unit_vector
                cube_imagePoints[2,:] -= delta*unit_vector
            else:
                print("Fix fail!!!!!!!!!!!!!!!!!!!")
                break
            
            try_counter+=1
        
                # cv2.putText(img2, f"error", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
        if is_predict_fail:
            cv2.imshow("result",frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            continue

        xy_line_points=np.float32([[0,y,0],[x,y,0],[x,0,0],[x,y,z]]).reshape(-1,3)
        xy_line_points_img,_=cv2.projectPoints(xy_line_points,table_rvec,table_tvec,mtx,dist)
        img2 = draw_xy_lines(img2,xy_line_points_img)
        img2 = draw_z_lines(img2,xy_line_points_img)

        axis = np.float32([[25,0,0], [0,25,0], [0,0,25]]).reshape(-1,3)
        axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
        axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
        img2=draw_axis(img2,cube_imagePoints.astype(int),axis_cube_img_points)
        img2=draw_axis(img2,table_imagePoints.astype(int),axis_table_img_points)


        # cv2.circle(img2, cube_imagePoints[2,:].astype(np.int16).ravel(),radius, (255,255,0), 2)
        cv2.circle(img2, (centroid_X,centroid_Y),radius, (255,0,255), 2)
        cv2.circle(img2, predict_cube_center_imagePoint.ravel(),5, (255,0,255), -1)
        cv2.putText(img2, f"{predict_cube_center_imagePoint}", (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 255), 2)
        cv2.putText(img2, f"{x=:.1f}, {y=:.1f}, {z=:.1f}", (10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        if np.linalg.norm(np.array((centroid_X,centroid_Y))-predict_cube_center_imagePoint)>radius:
            cv2.putText(img2, f"error", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
        else:
            cv2.putText(img2, f"fine", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
            cv2.putText(img2, f"{z=:.1f} {try_counter=}", (10,300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4)
            print(f"{z=}{try_counter=}")
        cv2.imshow("result",img2)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else :
        break
cv2.destroyAllWindows()

In [ ]:
points = np.array([[[342,316]], [[403, 315]], [[371 ,370]], [[370 ,338]]]).reshape(4,2)
point=np.array([224,448])
distances = np.linalg.norm(points - point, axis=1)
nearest_index = np.argmin(distances)
point = points[nearest_index]
print(point)

### 報告用

In [ ]:


with open('camera/calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)
with open('table_points.pkl','rb') as file:
    table_objectPoints,table_imagePoints = pickle.load(file)
with open('table_solvePnP.pkl','rb') as file:
    table_rvec,table_tvec = pickle.load(file)
color = "green"
while cap.isOpened():
    cap_success, frame = cap.read()
    if cap_success:
        height =480
        scale = height / frame.shape[0]
        frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        img2  = detector2.detect(frame,index=None,color=None,show_process_img=True)

        cube_imagePoints=detector2.get_cube_largest_surface_imagePoints(color)
        if cube_imagePoints is None:
            cv2.imshow("result",frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            continue
        cube_objectPoints=np.array([(0,0,0),(0,25,0),(25,25,0),(25,0,0)])

# print(cube_imagePoints)
        PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints.astype(float),cube_imagePoints,mtx,dist)
# ret,table_rvecs,table_tvecs = cv2.solvePnP(table_objectPoints,table_imagePoints,mtx,dist)


        if PnP_success:
            R1, _ = cv2.Rodrigues(cube_rvec)
            R2, _ = cv2.Rodrigues(table_rvec)
            T1 = np.eye(4) 
            T2 = np.eye(4) 
            T1[:3, :3], T1[:3, 3] = R1, cube_tvec.T  # 方塊座標系旋轉加平移到相機座標系
            T2[:3, :3], T2[:3, 3] = R2, table_tvec.T # 桌面座標系旋轉加平移到相機座標系
            T2=np.linalg.inv(T2) # 取逆矩陣，變成 相譏座標系到桌面座標系
            transform_matrix = T2 @ T1  
            transformed_point = transform_matrix @ np.array([[12.5, 12.5, -12.5, 1]]).T
            # 输出结果
            print("物體在桌面座標系下的座標:\n", transformed_point)

        x,y,z = transformed_point[0:3].flatten()
        xy_line_points=np.float32([[0,y,0],[x,y,0],[x,0,0]]).reshape(-1,3)
        xy_line_points_img,_=cv2.projectPoints(xy_line_points,table_rvec,table_tvec,mtx,dist)
        img2 = draw_xy_lines(img2,xy_line_points_img)

        axis = np.float32([[25,0,0], [0,25,0], [0,0,25]]).reshape(-1,3)
        axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
        axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
        img2=draw_axis(img2,cube_imagePoints.astype(int),axis_cube_img_points)
        img2=draw_axis(img2,table_imagePoints.astype(int),axis_table_img_points)

        predict_cube_center_imagePoint = cv2.projectPoints(np.float32([x,y,z]),table_rvec,table_tvec,mtx,dist)
        predict_cube_center_imagePoint = predict_cube_center_imagePoint[0].astype(np.int16)
        
        contour_outer =detector2.get_cube_contour_outer(color)
        radius = int(0.04*cv2.arcLength(contour_outer,True))
        print(radius)
        M = cv2.moments(contour_outer)
        if M["m00"] != 0:
            centroid_X = int(M["m10"] / M["m00"])# 算形心x
            centroid_Y = int(M["m01"] / M["m00"])# 算形心y
        img2 = cv2.circle(img2, (centroid_X,centroid_Y),radius, (255,0,255), 2)
        img2 = cv2.circle(img2, predict_cube_center_imagePoint.ravel(),5, (255,0,255), -1)
        cv2.putText(img2, f"{predict_cube_center_imagePoint}", (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 255), 2)
        cv2.putText(img2, f"{x=:.1f}, {y=:.1f}, {z=:.1f}", (10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        if np.linalg.norm(np.array((centroid_X,centroid_Y))-predict_cube_center_imagePoint)>radius:
            cv2.putText(img2, f"error", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
        else:
            cv2.putText(img2, f"fine", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
        cv2.imshow("result",img2)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else :
        break
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('cube_surface_seg2.pt')

# Open the video file
cap = cv2.VideoCapture(0)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    PnP_success, frame = cap.read()

    if PnP_success:
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
model = YOLO("cube_surface_seg.pt")
img = cv2.imread("pnp_test_img/img2.png")
# 定义要添加的边框大小（以像素为单位）
height =480
scale = height / img.shape[0]
img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR) 

result = model.predict(img)
for r in result:
    print(r.probs.top5)
# print(result[0].cpu()[0])
values=[]
for mask in result[0].masks:
    test_value = np.array((mask.xyn))
    test_value[:,:,0]=test_value[:,:,0]*img.shape[1]
    test_value[:,:,1]=test_value[:,:,1]*img.shape[0]
    values.append(test_value.astype(int))
mask = np.zeros((img.shape[0],img.shape[1]), dtype=np.uint8)
mask = cv2.drawContours(mask,values,-1, 255, 1)
# print(test_value)
# img2 = result[0].plot()
cv2.imshow("img2",mask)


cv2.waitKey(0)
cv2.destroyAllWindows()


# 找桌面座標

## V1

In [ ]:
import cv2

# 创建一个空白图像

image = cv2.imread('pnp_test_img/img0.png')  # 替换为您的图像文件路径

drawing_image= image.copy()
Adjusted_image= None

isAdjusting=False
isDrawing = True  # 标记是否正在绘制线段
isFirstDraw=True
square_image_points = []
line_thickness=2
# 回调函数，用于处理鼠标事件
def draw_line(event, x, y, flags, param):
    global  isDrawing,drawing_image,Adjusted_image,square_image_points,isAdjusting,isFirstDraw,line_thickness
    if event == cv2.EVENT_LBUTTONDOWN:  # 鼠标左键按下事件
        if isDrawing:
            if isFirstDraw:
                isFirstDraw = False
            else :
                cv2.line(drawing_image,square_image_points[-1],(x,y),(255,0,0),2)
            if len(square_image_points) < 4:
                square_image_points.append((x,y))
            if len(square_image_points)==4:
                isDrawing = False
                isAdjusting = True
                Adjusted_image=image.copy()
                cv2.polylines(Adjusted_image, [np.array(square_image_points)], isClosed=True, color=(0, 255, 0), thickness=2)
                
        elif isAdjusting:
            distances = np.linalg.norm(np.array(square_image_points) - np.array((x,y)), axis=1)
            nearest_index = np.argmin(distances)
            square_image_points[nearest_index]=(x,y)
            Adjusted_image=image.copy()
            cv2.polylines(Adjusted_image, [np.array(square_image_points)], isClosed=True, color=(0, 255, 0), thickness=2)
    elif event == cv2.EVENT_MOUSEMOVE:  # 鼠标移动事件
        if isDrawing and not isFirstDraw:
            temp_image = drawing_image.copy()  # 复制图像以防止绘制覆盖
            cv2.line(temp_image, square_image_points[-1], (x, y), (0, 255, 0), line_thickness)
            cv2.imshow('Set Table axis', temp_image)
        if isAdjusting:
            cv2.imshow('Set Table axis', Adjusted_image)


# 创建一个窗口并将鼠标事件处理函数绑定到它
cv2.namedWindow('Set Table axis')
cv2.setMouseCallback('Set Table axis', draw_line)
# 在窗口中显示图像，并等待用户按下ESC键退出
while True:
    if not isAdjusting:
        cv2.imshow('Set Table axis', drawing_image)
    else:
        cv2.imshow('Set Table axis', Adjusted_image)
    key = cv2.waitKey(1)
    if key == 27:  # 如果按下ESC键，退出循环
        break
   
cv2.destroyAllWindows()


with open('table_points.pkl', 'wb') as file:
        pickle.dump((np.float32([(0,0),(custom_width,0),(custom_height,custom_width),(0,custom_height)]),
                     np.float32(square_image_points)), file)

# with open('table_points.pkl', 'rb') as file:
#     img_points,object_points = pickle.load(file)
#     print(img_points)
#     print(object_points)

# V2

In [ ]:
#超屌抓桌面程式


import cv2

#讀取寬高參數
square_height,square_width = setHeightWin()
square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])

with open('camera/calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)

image = cv2.imread('pnp_test_img/img101.png')  # 載入圖片

drawing_image = image.copy()
Adjusted_image = None 

isDrawing = True
isAdjusting = False
isFirstDraw = True
square_image_points = []
line_thickness = 2
# 回調函數，處理滑鼠事件
def draw_line(event, x, y, flags, param):
    global line_start, line_end, isDrawing,drawing_image,Adjusted_image,square_image_points,isAdjusting,isFirstDraw,line_thickness
    #偵測事件
    if event == cv2.EVENT_LBUTTONDOWN:  # 當按下左鍵
        if isDrawing:
            if isFirstDraw:
                isFirstDraw = False
            else :
                cv2.line(drawing_image,square_image_points[-1],(x,y),(255,0,0),line_thickness)
            if len(square_image_points) < 4:
                square_image_points.append((x,y))
            if len(square_image_points)==4:
                isDrawing = False
                isAdjusting = True
                frame_update()
        elif isAdjusting:
            distances = np.linalg.norm(np.array(square_image_points) - np.array((x,y)), axis=1)
            nearest_index = np.argmin(distances)
            square_image_points[nearest_index]=(x,y)
            frame_update()
    elif event == cv2.EVENT_MOUSEMOVE:  # 當滑鼠移動
        if isDrawing and not isFirstDraw:
            temp_image = drawing_image.copy()  # 複製圖像而非變更原圖
            cv2.line(temp_image, square_image_points[-1], (x, y), (0, 255, 0), line_thickness)
            cv2.imshow('Set Table axis', temp_image)
        if isAdjusting:
            cv2.imshow('Set Table axis', Adjusted_image)

    elif event == cv2.EVENT_MOUSEWHEEL: # 當滑鼠滾輪滾動
        delta = flags
        if delta > 0:
            if line_thickness==3:
                pass
            else:
                line_thickness+=1
        elif delta < 0:
            if line_thickness == 1:
                pass
            else:
                line_thickness-=1
        frame_update()

# 創建視窗
cv2.namedWindow('Set Table axis')
# 綁定監聽滑鼠事件，綁定處理事件回調函數
cv2.setMouseCallback('Set Table axis', draw_line)

def frame_update(): # 進行畫面更新
    global Adjusted_image, square_image_points,line_thickness,rvecs,tvecs
    Adjusted_image=image.copy()
    cv2.polylines(Adjusted_image, [np.array(square_image_points)], isClosed=True, color=(0, 255, 0), thickness=line_thickness)
    for coordinate,point in zip(square_object_points,square_image_points):
        cv2.putText(Adjusted_image, f"{list(coordinate)}", point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 255), 2)
    square_img_points = np.float32(square_image_points)
    # print(square_img_points)
    ret,rvecs,tvecs = cv2.solvePnP(square_object_points,square_img_points,mtx,dist)
    # print(rvecs)
    axis = np.float32([[100,0,0], [0,100,0], [0,0,100]]).reshape(-1,3)
    axis_img_points, _ = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    draw_axis(Adjusted_image,square_img_points.astype(int),axis_img_points)

# 顯示圖像
while True:
    if not isAdjusting:
        cv2.imshow('Set Table axis', drawing_image)
    else:
        cv2.imshow('Set Table axis', Adjusted_image)
    key = cv2.waitKey(1)
    if key == 27:  # 當按下ESC退出循環
        break
    if len(square_image_points)!=4: #確認是否四個像素座標點抓到了
        continue
    if key == ord("r"): # Z軸對調
        square_image_points[1],square_image_points[3]=square_image_points[3],square_image_points[1]
        # square_coordinates[1],square_coordinates[3]=square_coordinates[3],square_coordinates[1]
        frame_update()
    if key == ord("e"): # 變更原點
        square_image_points=square_image_points[-1:]+ square_image_points[:-1] #前包後不包
        frame_update()
    if key == ord("w"): # 重設長寬
        square_height,square_width = setHeightWin()
        square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
        frame_update()
    if key ==ord("q"): # 長寬互換
        square_height,square_width=square_width,square_height
        square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
        frame_update()
    if key == ord("s"): # 儲存真實與圖像座標和PNP計算結果
        with open('table_points.pkl', 'wb') as file:
                pickle.dump((square_object_points,np.float32(square_image_points)), file)
        with open('table_solvePnP.pkl', 'wb') as file:
                pickle.dump((rvecs,tvecs), file)
        print("saved")
                    
cv2.destroyAllWindows()

In [ ]:
import tkinter as tk

# 創建獲取長寬表單視窗
def setHeightWin():
    root = tk.Tk()
    root.title("Input Form")

    # 獲取螢幕寬高
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # 設置視窗寬高(像素)
    window_width = 400  
    window_height = 150 

    # 計算視窗左上角座標
    x = (screen_width - window_width) // 2
    y = (screen_height - window_height) // 2

    # 視窗生成
    root.geometry(f"{window_width}x{window_height}+{x}+{y}")

    # 創建label, entry元件顯示讀取參數
    label_height = tk.Label(root, text="Height:", font=("Helvetica", 14))
    label_height.pack()

    entry_height = tk.Entry(root, font=("Helvetica", 14),justify="center")
    entry_height.pack()

    label_width = tk.Label(root, text="Width:", font=("Helvetica", 14))
    label_width.pack()

    entry_width = tk.Entry(root, font=("Helvetica", 14),justify="center")
    entry_width.pack()
    
    result = []
    # 處理送出的表單
    def submit_form():
        height = entry_height.get()
        width = entry_width.get()
        result.extend([height, width])

        if any(item == "" for item in result):# 當缺少參數的時候就將參數清空，不採取動作
            result.clear()
            return
        # 參數沒問題就銷毀視窗，回傳數值
        root.destroy()
    # 將enter綁定送出表單
    root.bind('<Return>', lambda event=None: submit_form())
    # 創建"送出"按鈕
    submit_button = tk.Button(root, text="送出", command=submit_form,width=10,height=1,font=("Helveticsa",14))
    submit_button.pack()

    # 啟動Tkinter主循環
    root.mainloop()
    return result